
# Goal: 

1. Create an anime recomendation system via webscraping data from myanimelist and creating my own database to use for Machine learning models.
2. Publish a working application for online users
3. Create something for my portfolio and learn more about the data & coding space.






# Plan

1. Data scraping & Pre-processing
2. Hyperparameter Tuning
3. Model Training and prediction
4. Post processing
5. Evaluation


# User ratings data

- We want to try get 1k user data in order to create our user-rating matrix
- Format file to be User_id, Anime_id, user_rating

In [1]:
import os
from bs4 import BeautifulSoup
import requests
import time
from datetime import datetime
import json
from zipfile import ZipFile
import pandas as pd
import shutil
import zipfile
import random
import numpy as np

In [2]:
# Script data in elements wont show user information, such as user_id,register_date,anime_completed etc.
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException

In [3]:
driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32/chromedriver.exe')
wait = WebDriverWait(driver, 10)

<ipython-input-3-f47c47b3b5ba>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32/chromedriver.exe')


# Anime database

In [28]:
# Using selenium autoscript to webscrap all pages of MAL to compare anime dataset against jikan (earlier)
# Webscrap tv anime

def Anime_data_links(name='tv'):
    
    """
    Takes in an input of either 'tv' or 'movie', to specify if you want to web scrape from either type of anime, default is
    set to 'tv'. This function will return a list of links for each anime in that category.
    
    """
    
    driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32/chromedriver.exe')
    wait = WebDriverWait(driver, 10) # Time our webdriver takes befiore timing out - stoping script if error during web scraping.

    anime_links = []
    for page in range(0,25000,50):
        time.sleep(1) # Time stop between each page, so not to overload server and risk an ip ban.
        driver.get(f'https://myanimelist.net/topanime.php?type={name}&limit=' + str(page))
    # Find all elements in page by CSS selector's element & save as a link variable
        link = driver.find_elements(By.CSS_SELECTOR, 'div[class="detail"] h3 a')
        for item in link:
            anime_links.append(item.get_attribute('href'))
            
    return anime_links

In [7]:
"""
Break our links into equal sized chunks, so its easier to run/complete.

"""
def seperate(lst,n):
    
    for i in range(0,len(lst),n):
        yield lst[i:i + n]



In [8]:
sep_anime = list(seperate(,1000))

In [9]:
# converted to list and seperated by 1000 links for both tv and movie anime.

test_tv_list1 = sep_tv[0]
test_tv_list2 = sep_tv[1]
test_tv_list3 = sep_tv[2]
test_tv_list4 = sep_tv[3]
test_tv_list5 = sep_tv[4]

test_movie_list1 = sep_movie[0]
test_movie_list2 = sep_movie[1]
test_movie_list3 = sep_movie[2]

# Note

Since the code below is in function form, it would work best if you loaded in the anime links in the form of a csv than convert that into a list and then use that list as inputs for each function.

In [26]:
# Script to web scrape based off list of links for both movies and tv anime.

def Anime_database(anime_links):
    
    """
    Takes a list of anime links and webscraps each individual anime for its title,genre,rating,url,source and studio.
    
    and then returns multiple lists of the webscraped elements.
    """

    driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32/chromedriver.exe')
    wait = WebDriverWait(driver, 10)

    titles = []
    ratings = []
    genres = []
    errors = []
    links = []
    studios = []
    sources = []
    episodes = []
    members = []
    demographics = []

    for item_link in anime_links:
        time.sleep(1)
        driver.implicitly_wait(5) # Time driver should wait for when searching for an element not immediately present.
        driver.get(item_link)

        if requests.get(item_link).status_code == 404:
            continue


        # Some anime pages dont contain the genre element,so we must check, thats why we specified implicitly_wait above.
        elif driver.find_elements_by_xpath('//*[contains(text(), "Genre")]/parent::div'):
            url = item_link
            genre = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[contains(text(), "Genre")]/parent::div')))
            title = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[class="h1-title"]')))
            rating = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[class*="score-label"]')))
            studio = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[contains(text(), "Studios")]/parent::div')))
            source = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[contains(text(), "Source")]/parent::div')))
            episode = wait.until(EC.visibility_of_element_located
            member = wait.until(EC.visibility_of_element_located
            demographic = wait.until(EC.visibility_of_element_located

            links.append(url)
            genres.append(genre.text)
            titles.append(title.text)
            ratings.append(rating.text)
            studios.append(studio.text)
            sources.append(source.text)

        else:
            name = driver.find_element_by_css_selector('div[class="h1-title"]')
            errors.append(name)
            continue
            
    return titles,ratings,genres,errors,links,studios,sources

In [18]:
df = pd.DataFrame(
{
    'Url': links,
    'Title':titles,
    'Rating':ratings,
    'Genre':genres,
    'Studio': studios,
    'Source': sources
})

In [19]:
df.to_csv('anime_data_movie3.csv')

# Users

In [3]:
# Function for Webscraping Users who were recently online MAL

# Passed

def Get_Active_Users(pages = 10):
    
    """ 
    This will scrape 25 names per page, 
    So specify how many pages you would like to scrap from as the argument. (Default is 10 pages)
    Returns a list of lists, each list contains 20 names.
    """
    import requests
    from lxml import html
    import numpy as np
    
    user_names = []
    
    for number in range(0,pages):
        link = 'https://myanimelist.net/users.php?lucky='+ str(number)
        page = requests.get(link)
        tree = html.fromstring(page.content)
        names = tree.xpath('//td[@align="center"]/div/a/text()')
        user_names.append(names)
        time.sleep(0.5)
        
    return user_names

In [4]:
# Use this function after webscraping user_names list of lists.

def flatten(_list):
    """Applies list comprehension to a given list of lists."""
    _list = [item for name in user_names for item in name]
    return _list

In [5]:
# Convert to CSV

def Data_to_csv(data):
    """ Converts an array of data into a Dataframe --> and then saved as a CSV."""
    import pandas as pd
    
    df = pd.DataFrame(data)
    return df.to_csv(f'{data}.csv')

In [3]:
list_names = pd.read_csv('User_names.csv')

In [6]:
list_names = list_names['User_Names'].tolist()

In [7]:
list_names

['Wan_Gui',
 'TrumpetMonkey',
 'Jeppie2001',
 'KodipSuradip',
 'Berato',
 'ElrondSeregon',
 'Roofofthexar',
 'Deima82',
 'ghostyl',
 'MangoBerry69',
 'DenWolf',
 'genismaig',
 '_Lyra',
 'jevonooi',
 'Its_JohnPi',
 'Alice-kaigaineki',
 'Shpaku',
 'Corn-Eater-Man',
 'BronzeUpp',
 'md10rts',
 'koalaOtaku077',
 'OtakuSam47',
 'ghost45thick',
 'KoHarve',
 'Foxeiro',
 'SpockyDude',
 'lolLaylaa',
 'Katzedan',
 'yaichikun40',
 'shambl',
 'Pablox1422',
 'BobbyBharath',
 'Hanz1_0wy',
 'JustTehc',
 'asdwqezxc',
 '_irahima_',
 'Prats27',
 'Yujio25',
 '#NAME?',
 'Real_Omega_',
 'Pentabra',
 '_Xolotl',
 'SAbeerBOY',
 'Rre_ree',
 'Disappointment17',
 'Hiken_no_s',
 'Anime134340',
 'aspirat2110',
 'iLMAO',
 'cutencuddly',
 'eplipswich',
 'ewiin',
 'Franklin_elaina',
 'DaakAMQ',
 'KingOfRandomness',
 'homu95',
 'x4163',
 'evilturkey5',
 'Khaeldyr',
 'LoliGoddd',
 'Richter_Belmont',
 'Roguevashda',
 'Fousch',
 'SamurEye',
 'Revali420',
 'Luffy8055',
 'Tafseer786',
 'LordFire00',
 'M1DN1GH7',
 '69Piece',

# User List

In [29]:
def user_data(list_names):
    
    driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32/chromedriver.exe')
    wait = WebDriverWait(driver, 20)
    SCROLL_PAUSE_TIME = 3
    
    User_Rating_Matrix = {}
    
    for name in list_names:
        
        reached_page_end = False
        
        url = f'https://myanimelist.net/animelist/{name}?status=2'
        driver.implicitly_wait(10) # Time driver should wait for when searching for an element not immediately present.
        driver.get(url)
        
        if requests.get(url).status_code == 504:
            continue
        
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while not reached_page_end:
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            if new_height == last_height:
                reached_page_end = True  
            else:
                last_height = new_height
                
        title = driver.find_elements_by_xpath("//td[@class='data title clearfix']//a[@class='link sort']")        
        rating = driver.find_elements_by_class_name('score')
        
        user_list = {}

        for (anime_name,score) in zip(title,rating[1:]):
            user_list[anime_name.get_attribute('textContent')] = score.text

        User_Rating_Matrix[name] = user_list
    
    return User_Rating_Matrix

In [47]:
# This version of the function will get data as an array format like name-anime_title-rating format

#Todo create another function like this BUT,use Async/wait to make multiple requests simultaneously. This will reduce script
# Run time


def user_data(list_names):
    
    import numpy as np
    
    driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32/chromedriver.exe')
    wait = WebDriverWait(driver, 20)
    SCROLL_PAUSE_TIME = 3
    
    data_list = []
    
    for name in list_names:
        
        reached_page_end = False
        
        url = f'https://myanimelist.net/animelist/{name}?status=2'
        driver.implicitly_wait(10) # Time driver should wait for when searching for an element not immediately present.
        driver.get(url)
        
        if requests.get(url).status_code == 504:
            continue
        
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while not reached_page_end:
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            if new_height == last_height:
                reached_page_end = True  
            else:
                last_height = new_height
                
        title = driver.find_elements_by_xpath("//td[@class='data title clearfix']//a[@class='link sort']")        
        rating = driver.find_elements_by_class_name('score')
        
        for (anime_name,anime_rating) in zip(title,rating[1:]):
            data_list.append([name,anime_name.get_attribute('href'),anime_rating.text])

        data_array = np.array(data_list)
            
    return data_array

In [17]:
import asyncio
from aiohttp import ClientSession
import pathlib

async def main():
    url = 'https://myanimelist.net/animelist/hollow12?status=2'
    html_body = ''
    async with ClientSession() as session:
        async with session.get(url) as response:
            html_body = await response.read()
    return html_body

html_data = await main()
output_dir = pathlib.Path().resolve() / 'snapshots'
output_dir.mkdir(parents=True, exist_ok=True)
output_file = output_dir / '2022.html'
output_file.write_text(html_data.decode())
#with open('C:\Users\Terry\Portfolio 2022', 'w') as f:
    #f.write(html_data.decode())


786776

In [18]:
anime_links = pd.read_csv('links_tv.csv')

In [22]:
anime_links = anime_links['url'].to_list()

In [ ]:
class preprocessing:
    
    def __init__(self,df):
        self.df = df
    
    def first_step(self):
        self.df = #function
        return self
    
    def second_step(self):
        self.df = #function
        return self
    
    def get_df(self):
        return self.df

In [48]:
data = user_data(list_names)

<ipython-input-47-132e09fe00c9>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32/chromedriver.exe')
<ipython-input-47-132e09fe00c9>:42: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  title = driver.find_elements_by_xpath("//td[@class='data title clearfix']//a[@class='link sort']")
<ipython-input-47-132e09fe00c9>:43: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  rating = driver.find_elements_by_class_name('score')


In [52]:
len(data)

353069

In [53]:
df = pd.DataFrame(data,columns=['User_Name','Anime_Id','User_Rating'])

In [57]:
df.to_csv("User_animelists_v2.csv")

In [ ]:
data = user_data(list_names)

In [ ]:
User_Animelists = pd.DataFrame.from_dict(data, orient='index')

In [ ]:
User_Animelists.to_csv('User_Animelists.csv')

# Note: Missing names

- Users with heavily formatted anime lists or private lists, cannot be included in my final list of user_names because of diffculties involving html layout changes to a persons animelist, which affect how the data is displayed, this is only a small portion of the inital user_name data, roughly 15% or 300 names.

- It is difficult to solve this with automated webscraping with my current skills because, each user which was excluded exlcusively due to animelist formatting, has made drastic changes to their list by directly adjusting the front end code and really customising their lists, and since every user can be a unique case of how they approached this change, i feel its a lot of effort to solve, for sometihng only a very amount of people actually commit to changing.

# Problem

- We have run into formatting issues, currently our script just makes a list of list.
- score = [ [user_list1],[user_list2] ] and so on.
- When converted to DF format, its a mess and i cannot tidy it easily.
- I shouldnt be using list.append for such a large amount of data. I have to utilise something else.

TDS: Efficiency is just as important as functionality!
You need to understand that even if there are multiple tools that can get the job done, some are simply superior to others. Learning which data structure is ideal for a specific scenario will serve as an immense step for enhancing the quality of your code.

While lists certainly are a reliable data structure, they are not the most fitting for every case. Here are a few alternatives that are worthy of consideration.

1. Tuples
2. Numpy Arrays
3. Sets
4. Dictionary

# Solved:

- Create a dictionary of dictionaries where each user_id holds a dictionary of an anime title and its related rating.
- Created a function which webscraps MAL user_animelist data and returns a dictionary of dictionaries

In [ ]:
user_data()

# Problem

- Users with heavily formatted anime lists, when i run find_element by class( "title" ), i get the result of add - more instead of all the anime names.

# Solved

- The problem is, for this specific use-case, selenium cannot grab the anime title because these users have formatted their animelist, so that you must hover over the anime in order for the title to show.

- Selenium works, by selecting the element on the page directly and inspecting it. If the anime title isnt shown because of the persons 0 opacity, than selenium cannot inspect and find the anime title, hence it returns an empty or add-more result.

- Specify by Xpath will solve this issue.

In [31]:
#title = driver.find_elements_by_class_name('data.title.clearfix')
title = driver.find_elements_by_xpath("//td[@class='data title clearfix']//a[@class='link sort']")

<ipython-input-31-8baa616ec5a4>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32/chromedriver.exe')
<ipython-input-31-8baa616ec5a4>:33: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  title = driver.find_elements_by_xpath("//td[@class='data title clearfix']//a[@class='link sort']")
<ipython-input-31-8baa616ec5a4>:35: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  rating = driver.find_elements_by_class_name('score')


In [32]:
title[5].get_attribute('textContent')

'Attack on Titan'

# Problem:
- We want to scroll infinitely down on a persons animelist, untill it has loaded every anime title they have seen.
- Certain factors, like MAL only preloading 300 titles per scroll down. Some users have 1000+ titles on their lists.

CS Logic:

1. capture page height before scrolling down;
2. scroll down using key down;
3. capture page height after scrolling down;
4. if page height was same before and after scrolling, stop scrolling

# Solved

- Implemented an infinite scroll while loop, by comparing page height before and after content load and scrolling down if heights arent equal.

In [144]:
reached_page_end = False
        
SCROLL_PAUSE_TIME = 5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while not reached_page_end:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        reached_page_end = True
    else:
        last_height = new_height

<ipython-input-144-b9b50d863021>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32/chromedriver.exe')
<ipython-input-144-b9b50d863021>:33: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  title = driver.find_elements_by_xpath("//td[@class='data title clearfix']//a[@class='link sort']")
<ipython-input-144-b9b50d863021>:34: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  rating = driver.find_elements_by_class_name('score')



# Problem
- Find the user_id class somewhere to extract the element.

- We can try to specify the attribute of the user-image class and take the data-src and then extract the users id in the src on the image.

# SOLVED

In [12]:
user_id = []
for name in list_names:
    r = requests.get(f'https://myanimelist.net/profile/{name}')
    content = r.content
    soup = BeautifulSoup(content, "html.parser")

    images = soup.findAll('img')[2]
    image_src = images.get("data-src")
    user_id.append(image_src)

https://cdn.myanimelist.net/images/userimages/3739545.jpg?t=1666746000


# TODO:

- On our Excel file for User_animelist, append user_ids using profile pictures to user_name in excel
- Replace anime titles on User_animelist with MAL_ids for each anime, using Full_anime_Data_part2, by matching titles and then assigning MAL id.


In [3]:
# Our finished User_name csv has 300 less names, because we were unable to webscrap some users due to privacy, so i must specify
# The second column of user_names which have valid lists.

# Run this second if you want specify names which have passed our webscrap process.

list_names = pd.read_csv('User_names_filtered.csv')

# Should return 1629 names
len(list_names)

1628

In [4]:
list_names = list_names['user_name'].tolist()

In [5]:
list_names

['Wan_Gui',
 'aspirat2110',
 'Retardi',
 'Ferensi',
 'Ark-Hu',
 'frishzx',
 'BePee',
 'DaBigFridge',
 'bondpapers',
 'Walrsu',
 'Mareep-chan',
 'Touki02',
 'Terico',
 '_Anime_sucks',
 'TerraIllumina',
 'kyoshimai',
 'SoggySlopster',
 'evoneko',
 'Kzzz',
 'Kafaei',
 '_Alive_',
 'CarlosVic',
 'X500658',
 'AK25',
 'Jasik2',
 'Sakenz',
 'Orion_Gospel',
 'frild',
 'Archfiend836',
 'anime-prime',
 'N_J_N',
 'Alcowe',
 'nexuro01',
 'Nomolas',
 'DeonX',
 'based_pawl',
 'RodanG',
 'HarusakiYorokobi',
 'MonkeeDan',
 'Givemeanaccount',
 'Loomkoom',
 'Amonimus',
 'Hatreddd',
 'nanashi796',
 'Twintail_Daemon',
 'Jokobo',
 'nickg14',
 'GVG',
 'NagisaXpsionne',
 'IridescentJaune',
 'Matthew98gfx',
 'PostMAL',
 'chocomayu',
 'ghostyl',
 'Shpaku',
 'BronzeUpp',
 'SpockyDude',
 'Katzedan',
 'asdwqezxc',
 '_irahima_',
 'SAbeerBOY',
 'Rre_ree',
 'Hiken_no_s',
 'eplipswich',
 'KingOfRandomness',
 'x4163',
 'Khaeldyr',
 'Roguevashda',
 'Fousch',
 'vlnandrei',
 'YoungFuzzy',
 'jnelsonit',
 'zakbrito',
 'Dani

# Problem

- I have to make sure that there is actually a profile picture to scrape, pretty sure if they dont upload a picture, it wont scrap it

- Also not all the data-src urls are profile pictures, some are just anime covers, so that means i need to be more explicit with my code.

# Solved

- If you want to check image is present or not under a section then you have to create a webelement for that section.

WebElement section= driver.findElement(By.xpath("//img(@class=‘Section')"));
Now create an image element under section element.

WebElement image= section.findElement(By.xpath("//img(@class=‘Test Image')"));
Now check image is exist or not.

boolean imagePresent = image.isDisplayed();
Now assert on boolean result.

assertTrue(imagePresent, “No image is exist”);

In [15]:
def user_id(names):
    
    driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32/chromedriver.exe')
    wait = WebDriverWait(driver, 20)
    
    user_ids = []
    
    for name in names:
    
        time.sleep(2)
        url = f"https://myanimelist.net/profile/{name}"
        driver.implicitly_wait(10) # Time driver should wait for when searching for an element not immediately present.
        driver.get(url)

        if requests.get(url).status_code == 404:
            user_ids.append(name)
            continue
        
        elif driver.find_element_by_class_name("user-image").text == 'No Picture':
            image = driver.find_element_by_class_name("user-image")
            user_ids.append(image.text)
            
        else:
            product_img_xpath = '//div[@class="user-profile"]//img'
            image = driver.find_element_by_xpath(product_img_xpath)
            user_ids.append(image.get_attribute('src'))
            
    return user_ids

In [16]:
user_id_data = user_id(list_names)

<ipython-input-15-268d35f676d3>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32/chromedriver.exe')
<ipython-input-15-268d35f676d3>:19: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  elif driver.find_element_by_class_name("user-image").text == 'No Picture':
<ipython-input-15-268d35f676d3>:25: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  image = driver.find_element_by_xpath(product_img_xpath)
<ipython-input-15-268d35f676d3>:20: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  image = driver.find_element_by_class_name("user-image")


In [17]:
user_id_data

['https://cdn.myanimelist.net/images/userimages/13212366.jpg?t=1668108600',
 'https://cdn.myanimelist.net/images/userimages/7468145.jpg?t=1668123600',
 'https://cdn.myanimelist.net/images/userimages/7256477.jpg?t=1667953800',
 'https://cdn.myanimelist.net/images/userimages/12292344.jpg?t=1668083400',
 'https://cdn.myanimelist.net/images/userimages/11971319.jpg?t=1668096000',
 'https://cdn.myanimelist.net/images/userimages/11968588.jpg?t=1668015600',
 'https://cdn.myanimelist.net/images/userimages/6433383.jpg?t=1668118800',
 'https://cdn.myanimelist.net/images/userimages/14472720.jpg?t=1668130800',
 'https://cdn.myanimelist.net/images/userimages/14143996.jpg?t=1668083400',
 'https://cdn.myanimelist.net/images/userimages/7008357.jpg?t=1668126000',
 'https://cdn.myanimelist.net/images/userimages/6015878.jpg?t=1668118800',
 'https://cdn.myanimelist.net/images/userimages/13615440.jpg?t=1668129600',
 'https://cdn.myanimelist.net/images/userimages/3768757.jpg?t=1668067800',
 'https://cdn.myan

In [18]:
len(user_id_data)

1628

In [19]:
user_id_data_df = pd.DataFrame(user_id_data)

In [20]:
user_id_data_df.to_csv("user_id_data.csv")

# Cleanup

- Use Excel & SQL to format and clean tables further, i.e matching columns on PK,combining tables. We will use power query and excel to tidy up the api scrapped data and then reload it into python to create our matrix.
- Some anime profiles are private, and therefore some profiles were not accessible from our filtered list, so we are missing their profile pictures, which means we are missing their user ids. So we must remove those users from our final dataset. We had 1628 user names filtered, now we have 
